## Baseline model
표 형식의 데이터를 다루는 머신러닝 파이브라인의 일반적인 순서는 아래와 같다.

Data Preprocessing - Feature Engineering - Model Training - test - Predict Result

## Data Preprocessing
* 결측값 -> 0으로 대체, 제품 보유여부의 정보가 없으면 보유하지 않음을 가정
* 훈련 데이터와 테스트 데이터의 병합(날짜변수로 쉽게 구분을 할 수 있음, 24개의 고객변수가 동일한 것을 그대로 병합하고 테스트에 없는 24개의 제품변수는 0으로 채움)
* 범주형 데이터는 factorize()를 통해 label encoding을 수행하고 데이터 타입이 object로 표현되는 수치형 데이터는 .unique()를 통해 특이값들을 대체하거나 제거하고 정수형 데이터로 변환한다.
* 추후, 모델 학습에 사용할 변수 이름을 features 리스트에 미리 담아둔다.

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

# 데이터를 불러옴
train = pd.read_csv('C:\\Users\\silve\\Desktop\\santander\\data\\train_ver2.csv')
test = pd.read_csv('C:\\Users\\silve\\Desktop\\santander\\data\\test_ver2.csv')

In [7]:
# 변수들을 별도로 저장해둠
prods = train.columns[24:].tolist()

# 제품 변수 결측값을 0으로 대체함
train[prods] = train[prods].fillna(0.0).astype(np.int8)

# 24개중 하나도 보유하지 않은 고객의 데이터를 제거
no_product = train[prods].sum(axis = 1) == 0
train = train[~ no_product]

# 훈련과 테스트 데이터를 통합하고, 테스트 데이터에 없는 제품의 변수는 0으로 대체하여 채운다
for col in train.columns[24:]:
    test[col] = 0
df = pd.concat([train,test], axis = 0)

# 학습에 사용할 변수를담는 list생성
features = []

# 범주형 변수를 .factorize()함수를 통해 label encoding함
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi',
                   'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel = -99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고 정수형으로 변환한다
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

#학습에 사용할 수치형 변수를 features에 추가한다.
features += ['age', 'antigue', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes', 'ind_actividad_cliente']

## Feature Engineering
feature engineering에서는 모델에서 사용할 파생 변수를 생성한다.
> baseline 모델에서는 24개의 고객변수와 4개의 날짜기반 파생변수 그리고 24개의 lag-1 변수를 사용한다.

결측값은 임시로 -99로 대체한다. (사이킷런에서 제공하는 머신러닝 모델은 결측값을 고치지 않으면 안되지만 xgboost모델에서는 결측값을 정상적으로 입력받는다.)


이밖에도 다양하게 파생변수를 생성하여 사용할수도 있다.

In [13]:
# 두 날짜 변수에서 연도와 월 정보를 추출한다
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 
                                             if x.__class__ is float
                                             else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0
                                         if x.__class__ is float
                                         else float(x.split('-')[0])).astype(np.int16)
features +=['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0
                                                 if x.__class__ is float
                                                 else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0
                                         if x.__class__ is float
                                         else float(x.split('-')[0])).astype(np.int16)

# 그 외의 변수 결측값은 -99로 대체
df.fillna(-99, inplace = True)

# lag-1 데이터 생성
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split('-')]
    int_date = (int(Y) - 2015)*12 + int(M)
    
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다,
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고 int_date에 1을 더하여 lag를 생성 후 변수에 _prev를 추가
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

# 원본데이터와 lag데이터를 ncodper와 int_date를 기준으로 합친다.
df_train = df.merge(df_lag, on = ['ncodpers', 'int_date'], how = 'left')

# 메모리 효율을 위해 변수를 제거
# del df, df_lag

# 결측값 채우기
for prod in prods:
    prev = prod + '_prev'
    df_train[prev].fillna(0, inplace = True)
df_train.fillna(-99, inplace = True)

# lag-1 변수를 추가
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

## Model training
2015-01-28 ~ 2016-05-28의 1년 6개월의 데이터가 훈련데이터로 훈련되고예측 데이터는 2016-06-28의 미래데이터이다. -> 내부 교차 검증 과정에서 2016-05-28의 데이터를 검증으로 분리하고 나머지를 훈련으로 분리하여 사용한다.

baseline모델에서는 2016-01-28 ~ 2016-04-28만을 훈련한다. 2016-05-28은 검증모델로 사용하였다.

#### XGBoost parameters
* max_depth: 값이 클수록 복잡하며 과적합의 원인이 될 수 있음
* eta: learning rate와 동일 값이 높으면 학습이 안될 수 있고 너무 낮으면 학습이 느려질 수 있다.
* colsample_bytree: 트리를 생성할 때 훈련 데이터에서 변수를 샘플링해주는 비율 -> 서로의 약점을 보완함. 대개는 0.6~0.9를 사용
* colsample_bylevel: 트리의 레벨 별로 훈련 데이터의 변수를 샘플링해주는 비율. 대개 0.6~0.9를 사용

In [14]:
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
train = df_train[df_train['fecha_dato'].isin(use_dates)]
test = df_train[df_train['fecha_dato'] == '2016-06-28']
del df_train

# 훈련데이터에서 신규 구매만 추출
x = []
y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    pr_x = train[(train[prod] == 1) & (train[prev] == 0)]
    pr_y = np.zeros(pr_x.shape[0], dtype = np.int8) + i
    x.append(pr_x)
    y.append(pr_y)
xy = pd.concat(x)
y = np.hstack(y)
xy['y'] = y

# 훈련, 검증 데이터로 분리
valid_date = '2016-05-28'
xy_train = xy[xy['fecha_dato'] != valid_date]
xy_valid = xy[xy['fecha_dato'] == valid_date]

In [32]:
# XGBoost 모델의 parameter를 설정
param = {'booster': 'gbtree',
        'max_depth': 8,
        'nthread': 4,
        'num_class': len(prods),
        'objective': 'multi:softprob',
        'silent': 1,
        'eval_metric': 'mlogloss',
        'eta': 0.1,
        'min_child_weight': 10,
        'colsample_bytree': 0.8,
        'colsample_bylevel': 0.9,
        'seed': 2018,}

x_train = xy_train[[feature for feature in features 
                    if feature in xy_train.columns]]
y_train = xy_train['y']
dtrain = xgb.DMatrix(x_train, label = y_train, feature_names = features)

x_valid = xy_valid[[features for feature in features 
                    if feature in xy_train.columns]]
y_valid = xy_valid['y'].values
dvalid = xgb.DMatrix(x_valid, label = y_valid, feature_names = features)

# xgboost 모델로 학습
watch_list = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(param, dtrain, num_boost_round = 1000, evals = watch_list, early_stopping_rounds = 20)

# 모델 저장
import pickle
pickle.dump(model, open('model/xgb.baseline.pkl', 'wb'))
best_ntree_limit = model.best_ntree_limit

ValueError: feature_names must be unique

In [41]:
dtrain = xgb.DMatrix(x_train, label = y_train, feature_names = list(set(features)))


ValueError: feature_names must have the same length as data